In [ ]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

import re

def parse_article(line):
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16).map(parse_article)
result = wiki.take(1)[0]

In [ ]:
for word in result[:50]:
    print word

In [ ]:
# convert capital to lowercase
wiki_lc = wiki.map(lambda a: [x.lower() for x in a])

# import numpy as np
# wiki_lc = wiki.map(lambda a: np.char.asarray(a).lower())

In [ ]:
# generate word pairs for all articles
import numpy as np
def generate_word_pairs(article):
    if len(article) < 2:
        return []
    else:
        _array = np.chararray(len(article) - 1, unicode=True)
        _array[:] = '_'
        result = np.core.defchararray.add(article[:-1], _array)
        result = np.core.defchararray.add(result, article[1:])
        return result
word_pairs = wiki_lc.flatMap(lambda a: generate_word_pairs(a))

In [ ]:
# map word_pair to (word_pair, 1) for count
word_pair_kv = word_pairs.map(lambda a: (a, 1))

In [ ]:
# count them
word_pair_counts = word_pair_kv.reduceByKey(lambda a, b: a + b)
word_pair_counts.take(10)

In [ ]:
# filter narodnaya_
narodnaya_filter = word_pair_counts.filter(lambda a: a[0].startswith('narodnaya_'))
narodnaya_filter.collect()